In [ ]:
import pandas as pd
import re
import numpy as np
from datetime import datetime, date
%matplotlib inline

In [ ]:
#Define general functions
# Convert times to minutes

def hms_to_s(s):
    t = 0
    for u in s.split(':'):
        t = 60 * t + int(u)
    return t
 
def hms_to_min(s):
  s=s.split(':')
  t = int(s[0])*60+int(s[1])*1 +int(s[2])/60
  return t
 
def ms_to_min(s):
  s=s.split(':')
  t = int(s[0])*1 +int(s[1])/60
  return t


def Clean_times(times):
    # Search for opening bracket in the time followed by
    # any characters repeated any number of times
    if re.search('\(.*', times):
        # Extract the position of beginning of pattern
        pos = re.search('\(.*', times).start()
        # return the cleaned time
        return times[:pos]
    else:
        # if clean up needed return the same time
        return times


def Split_times(times):
    split_re = re.compile('.*\((.*)\).*')
    # Search for opening bracket in the time followed by
    # any characters repeated any number of times
    if re.search('\(.*', times):
        # Split the times in group based on the pattern
        match = re.search(split_re, times)
         # return the cleaned time between parentheses
        return match.group(1)
    else:
        # if clean up needed return the same time
        return times
    
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%d:%02d:%02d" % (hour, minutes, seconds)

def hms_to_time(s):
    t = 0
    for u in s.split(':'):
        t = 60 * t + int(u)
    return convert(t)

def seconds_to_time(time_to_convert):
     return datetime.strptime(convert(time_to_convert), '%H:%M:%S').time()

In [ ]:
#df_2008 = pd.read_csv('uitslagen2008.csv')
df_2009 = pd.read_csv('uitslagen2009.csv')
df_2010 = pd.read_csv('uitslagen2010.csv')
df_2011 = pd.read_csv('uitslagen2011.csv')
df_2012 = pd.read_csv('uitslagen2012.csv')
df_2013 = pd.read_csv('uitslagen2013.csv')
df_2014 = pd.read_csv('uitslagen2014.csv')
df_2015 = pd.read_csv('uitslagen2015.csv')
df_2016 = pd.read_csv('uitslagen2016.csv')
df_2017 = pd.read_csv('uitslagen2017.csv')
years = [df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017]
results = pd.concat(years, sort=False)
results.info()

In [ ]:
# Clean up data frame by only keeping necessary columns
runners_df = results[["Place_y","Name","City_y","Country","AgegroupPlace","Agegroup","Guntime", "Chiptime", 
                      "5KM", "10KM", "15KM","20KM", "HALF", "25KM", "30KM", "35KM", "40KM", "Bib","Distance","Year"]]
runners_df.rename(columns={"Place_y": "Place","City_y": "City"},inplace=True)
runners_df['Year'] = runners_df['Year'].apply('{:.0f}'.format)
runners_df.head()

In [ ]:
#Clean and seperate columns
#difference between male and female agegroup. Femal agegroup stops at 50+ while male goes on to 60+
#Normalised to one agegroup 50+

runners_df['AgegroupPlace'] = runners_df['AgegroupPlace'].str.replace(r'\D', '').astype(int)
runners_df['Gender'] = runners_df['Agegroup'].str[0].replace({'V': 'F'})
runners_df['Age'] = runners_df['Agegroup'].str[1:].replace({'SR': '18-34', '35': '35-39', '40': '40-44', '45': '45-49','50':'50-','55':'50-','60':'50-'})
runners_df.head()


#df['Gender'] = df['Agegroup'].str[0]
#df['Age'] = df['Agegroup'].str[1:].map({
#    '014': '0-14',
#    '1524': '15-24',
#    '2534': '25-34',
#    '3544': '35-44',
#    '4554': '45-54',
#    '5564': '55-64',
#    '65': '65+'
#})

In [ ]:
splits = ("5KM","10KM","15KM","20KM","25KM","30KM","35KM","40KM")

for i in splits:
    runners_df[['distance_'+i+'_time','split_'+i+'_time']] = runners_df[i].apply(lambda x: pd.Series(str(x).split(" ")))
    runners_df['distance_'+i+'_time'] = runners_df['distance_'+i+'_time'].astype(str).apply(Clean_times)
    runners_df['split_'+i+'_time'] = runners_df['split_'+i+'_time'].astype(str).apply(Split_times)
    runners_df[['distance_'+i+'_time','split_'+i+'_time']] = runners_df[['distance_'+i+'_time','split_'+i+'_time']].replace({'nan':np.nan, 0:np.nan, "":np.nan}).fillna("00:00")

In [ ]:
#Convert all the time strings to a time in seconds
cols=[]
for i in splits:
    t1 = 'split_'+i+'_time'
    t2 = 'distance_'+i+'_time'
    cols.append(t1)
    cols.append(t2)
print(cols)
for i in cols:
    runners_df[i]= runners_df[i].apply(hms_to_s)
runners_df["Guntime_s"] = runners_df["Guntime"].apply(hms_to_s)
runners_df["Chiptime_s"] = runners_df["Chiptime"].apply(hms_to_s)
runners_df.head(10)

In [ ]:
#timing mat for splits at 10KM in 2012 was not working, so no (split)times for 10KM and no splittime for 15KM in 2012
#calculate timing based on even pace between 5K and 15K
#check if previous time is not NaN

runners_df[runners_df["Year"]=="2012"["split_10KM_time"] = runners_df[runners_df["Year"]=="2012"["distance_5KM_time"]
runners_df[runners_df["Year"]=="2012"["distance_10KM_time"] = runners_df[runners_df["Year"]=="2012"["distance_5KM_time"] + runners_df[runners_df["Year"]=="2012"["split_10KM_time"] 
runners_df[runners_df["Year"]=="2012"["split_15KM_time"] = runners_df[runners_df["Year"]=="2012"["distance_15KM_time"] - runners_df[runners_df["Year"]=="2012"
                                                                                                                                    
                                                                                                                                    
                                                                                                                                    ["distance_10KM_time"]

#Calculate last split
runners_df["split_42KM_time"] = runners_df["Chiptime"].apply(hms_to_s) - runners_df["distance_40KM_time"]
runners_df["distance_42KM_time"] = runners_df["Chiptime"].apply(hms_to_s)

   

In [ ]:
is2012 = runners_df[runners_df["Year"]=="2012"]
print(is2012.head())

In [ ]:
df_2012.describe()

In [ ]:
#drop all runners who also missed one or more timing mats

#first create NaN value for all the null, zero;s
for i in cols:
    runners_df[i]= runners_df[i].replace(0,np.nan)

#remove columns not needed or double
dataset_df = runners_df[["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s","Chiptime_s","HALF", 
                         "Bib","Gender","Age","Distance","Year",'split_5KM_time', 'distance_5KM_time', 'split_10KM_time', 'distance_10KM_time', 
                         'split_15KM_time', 'distance_15KM_time', 'split_20KM_time', 'distance_20KM_time', 'split_25KM_time', 
                         'distance_25KM_time', 'split_30KM_time', 'distance_30KM_time', 'split_35KM_time', 'distance_35KM_time',
                         'split_40KM_time', 'distance_40KM_time',"split_42KM_time","distance_42KM_time"]]
 
#drop all records with NaN    
dataset_df = dataset_df.dropna()

#Check drop percentage
dropped = ((len(runners_df) -len(dataset_df)) / len(runners_df))*100
print("percentage of runners dropped: "+ str(dropped))

In [ ]:
#Optional convert all the time in seconds to Datetime.Time series
#Difficult to perform calculations
col2 = ["split_5KM_time","split_10KM_time","split_15KM_time","split_20KM_time",
               "split_25KM_time","split_30KM_time","split_35KM_time","split_40KM_time","split_42KM_time","distance_5KM_time","distance_10KM_time","distance_15KM_time","distance_20KM_time",
               "distance_25KM_time","distance_30KM_time","distance_35KM_time","distance_40KM_time","distance_42KM_time"]
for i in col2:
    dataset_df[i] = dataset_df[i].apply(seconds_to_time)
dataset_df.head(10)

In [ ]:
df1 = dataset_df.melt(id_vars=["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s", "Chiptime_s", 
                      "HALF", "Bib","Gender","Age","Year"], 
              value_vars=["distance_5KM_time","distance_10KM_time","distance_15KM_time","distance_20KM_time",
               "distance_25KM_time","distance_30KM_time","distance_35KM_time","distance_40KM_time","distance_42KM_time"],
              var_name='distance', value_name='distance_time')
df2 = dataset_df.melt(id_vars=["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s", "Chiptime_s", 
                      "HALF", "Bib","Gender","Age","Year"], 
              value_vars=["split_5KM_time","split_10KM_time","split_15KM_time","split_20KM_time",
               "split_25KM_time","split_30KM_time","split_35KM_time","split_40KM_time","split_42KM_time"],
              var_name='distance', value_name='split_time')

In [ ]:
df1["distance"] = df1['distance'].str.extract('(\d+)', expand=False).astype(int)
df2["distance"] = df2['distance'].str.extract('(\d+)', expand=False).astype(int)

In [ ]:
df2[df2["Name"]=="Ivo Nobel"].head()

In [ ]:
df1 = df1.set_index(["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s", "Chiptime_s", 
                      "HALF", "Bib","Gender","Age","Year","distance", df1.groupby(["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s", "Chiptime_s", 
                      "HALF", "Bib","Gender","Age","Year","distance"]).cumcount()])
df2 = df2.set_index(["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s", "Chiptime_s", 
                      "HALF", "Bib","Gender","Age","Year","distance", df1.groupby(["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s", "Chiptime_s", 
                      "HALF", "Bib","Gender","Age","Year","distance"]).cumcount()])
df3 = (pd.concat([df1, df2],axis=1)
         .sort_index(level=14)
         .reset_index(level=14, drop=True)
         .reset_index())
df3.to_csv ("complete_dataset.csv", index = None, header=True)

In [ ]:
df3.head()

In the processed runners dataset, we can see duplicates of finish results, it’s because this table actually contains two types of observational units, marathon results and split times. To tidy it, we first generate identities for each marathon finish, i.e. id, and then separate them into different tables.

In [ ]:
#df = pd.read_csv('data/billboard-intermediate.csv')
df_marathon = df3[["Place","Name","City","Country","AgegroupPlace","Agegroup","Chiptime_s", "Guntime_s", 
                      "HALF", "Bib","Gender","Age","Year"]].drop_duplicates()
df_marathon.insert(0, 'id', range(1, len(df_marathon) + 1))
df_split = pd.merge(df3, df_marathon, on=["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s", "Chiptime_s", 
                      "HALF", "Bib","Gender","Age","Year"])
df_split = df_split[['id', 'distance', 'distance_time','split_time']]
df_marathon.to_csv('marathon-results.csv', index=False)
df_split.to_csv('marathon-splits.csv', index=False)
print(df_marathon, '\n\n', df_split)

In [ ]:
#number of finsihers
runners_by_country = df_marathon.groupby(['Year'])['Name'].count()
runners_by_country.plot.bar()

In [ ]:
#average finish time
runners_by_country = df_marathon.groupby(['Year'])['Chiptime_s'].mean()
runners_by_country.plot.line()

In [ ]:
import chartify
import bokeh
from bokeh.models import NumeralTickFormatter
from bokeh.models import LinearAxis

In [ ]:
runners_by_year = df_marathon.groupby(['Year'])['Name'].count().reset_index()
ch = chartify.Chart(blank_labels=True, x_axis_type='categorical')
ch.set_title("Finishers in the Rotterdam Marathon")
ch.set_subtitle("more and more")
ch.plot.bar(
        data_frame=runners_by_year,
        categorical_columns='Year',
        numeric_column='Name',
        categorical_order_by='labels',
        categorical_order_ascending=True)
ch.show()

In [ ]:
n_countries = 10
country_counts_sorted = df_marathon[['Country','Name']]
country_counts_sorted = country_counts_sorted.replace(country_counts_sorted.groupby(['Country'])['Name'].count().sort_values(ascending=False).index[n_countries:],'Other countries').groupby(['Country'])['Name'].count().sort_values(ascending=False).reset_index()

In [ ]:
ch = chartify.Chart(blank_labels=True, x_axis_type='categorical')
ch.set_title("International marathon")
ch.set_subtitle("finisher by country")
ch.plot.bar(
        data_frame=country_counts_sorted,
        categorical_columns='Country',
        numeric_column='Name',
        categorical_order_by='values')
ch.show()

In [ ]:
from math import pi
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

In [ ]:
data = country_counts_sorted.rename(columns={'Name':'value'})

In [ ]:
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(country_counts_sorted)]

p = figure(plot_height=350, title="International marathon", toolbar_location=None,
           tools="hover", tooltips="@Country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='Country', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
show(p)

In [ ]:
# Non-US countries: Does this follow some sort of power-trend law, i.e. the decrease between subsequent numbers?
country_counts_sorted = df_marathon[['Country','Name']]
country_counts_sorted_not_NED = country_counts_sorted.groupby(['Country'])['Name'].count().sort_values(ascending=False).reset_index()[1:]
country_counts_sorted_not_NED = country_counts_sorted_not_NED.rename(columns={'Name':'value'})
country_counts_sorted_not_NED = country_counts_sorted_not_NED.replace(country_counts_sorted_not_NED.groupby(['Country'])['value'].sum().sort_values(ascending=False).index[n_countries:],'Other countries').groupby(['Country'])['value'].sum().sort_values(ascending=False).reset_index()
data = country_counts_sorted_not_NED

data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(country_counts_sorted_not_NED)]

p = figure(plot_height=350, title="International marathon - non-dutch runners", toolbar_location=None,
           tools="hover", tooltips="@Country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='Country', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
show(p)

In [ ]:
runners_by_country_gender = df_marathon.groupby(['Year','Gender'])['Name'].count().reset_index()
ch = chartify.Chart(blank_labels=True, x_axis_type='categorical')
ch.set_title("Gender of the finishers")
ch.set_subtitle("More women are running the marathon")
ch.plot.bar_stacked(
        data_frame=runners_by_country_gender,
        categorical_columns='Year',
        numeric_column='Name',
        stack_column="Gender",
        normalize=True,
        categorical_order_by='labels',
        categorical_order_ascending=True)
ch.show()

In [ ]:
ch = chartify.Chart(blank_labels=True, y_axis_type='density')
ch.set_title("Histogram of the finishtimes")
ch.set_subtitle("2009 - 2017")
ch.plot.histogram(
        data_frame=df_marathon,
        values_column='Guntime_s',
        bins=40
        )
ch.figure
ch.axes.set_xaxis_label('Finish times')
ch.axes.set_xaxis_tick_values(list(range(0, 100000, 1800)))
ch.figure.xaxis.formatter = NumeralTickFormatter(format="00:00:00")
ch.show()

In [ ]:
ch = chartify.Chart(blank_labels=True, y_axis_type='density')
ch.set_title("Histogram of the finishtimes")
ch.set_subtitle("2009 - 2017")
ch.plot.histogram(
        data_frame=df_marathon,
        values_column='Guntime_s',
        color_column='Gender',
        bins=40
        )
ch.figure
ch.axes.set_xaxis_label('Finish times')
ch.axes.set_xaxis_tick_values(list(range(0, 100000, 1800)))
ch.figure.xaxis.formatter = NumeralTickFormatter(format="00:00:00")
ch.show()

In [ ]:
split_by_5KM = (
        df3.groupby(['distance','Gender'])['distance_time'].mean()
        .reset_index()  # Move 'date' from index to column
        )

In [ ]:
    ch = chartify.Chart(blank_labels=True, x_axis_type='linear')
    ch.set_title("Line charts")
    ch.set_subtitle("Plot two numeric values connected by an ordered line.")
    ch.plot.line(
        # Data must be sorted by x column
        data_frame=split_by_5KM.sort_values('distance'),
        x_column='distance',
        y_column='distance_time',
        color_column='Gender')
    ch.figure.yaxis.formatter = NumeralTickFormatter(format="00:00:00")
    ch.figure.xaxis.ticker = [5, 10, 15,20,25,30,35,40,42]
    ch.show()

In [ ]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
cf.go_offline()
box_age = df_marathon[['Year', 'Chiptime_s']]

fig = box_age.pivot(columns='Year', values='Chiptime_s').iplot(kind='box',asFigure=True)

#fig.update_layout(yaxis_tickformat = '%')
fig.iplot()

In [ ]:
boxplot = runners_df.boxplot(column=["split_5KM_time","split_10KM_time","split_15KM_time","split_20KM_time",
               "split_25KM_time","split_30KM_time","split_35KM_time","split_40KM_time"])